In [1]:
# Get the voxel centers
# Read them into a KD Tree
# Pull out a bunch based on the nearest gb found and write those to a file
# Check if the change in gb matches up by...entering point found in paraview?

In [2]:
# This will give the nearest voxel center (x,y,z) based on the voxel size being the length of 1 side of the voxel

def find_nearest_voxel_center(x, y, z, voxel_size):
    # This works because / gives a float while // gives an int. So this says that it's in one boundary until the next
    voxel_x = int(((x + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_y = int(((y + (voxel_size / 2)) // voxel_size) * voxel_size)
    voxel_z = int(((z + (voxel_size / 2)) // voxel_size) * voxel_size)
    return voxel_x, voxel_y, voxel_z

In [3]:
voxel_centers_ = []
grain_ids_ = []
natee = 0
with open('../../data/1-voxel-centers.csv', 'r') as voxel_centers_file:
    voxel_centers_file.readline()
    for line in voxel_centers_file:
        tokens = line.split(',')
        x = int(float(tokens[0]))
        y = int(float(tokens[1]))
        z = int(float(tokens[2]))
        grain_id = int(tokens[3])
        voxel_centers_.append((x, y, z))
        grain_ids_.append(grain_id)
        natee += 1
        if natee > 1000000:
            print('tick')
            natee = 0
print('done')

tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
tick
done


In [4]:
import numpy as np

voxel_centers = np.array(voxel_centers_)
grain_ids = np.array(grain_ids_)
print('done')

In [5]:
from scipy.spatial import cKDTree
# import sys
# sys.setrecursionlimit(10000)

kd_tree = cKDTree(voxel_centers)
print('done')

In [6]:
bx = 7.8821
by = 315.15
bz = 304.1

x, y, z = find_nearest_voxel_center(bx, by, bz, 2)
print(x,y,z)

fx = 0
fy = 304
fz = 316
actual_distance = ((x-fx)**2 + (y-fy)**2 + (z-fz)**2)**.5
print(actual_distance)

8 316 304
18.76166303929372


In [7]:
indices = kd_tree.query_ball_point((x,y,z),actual_distance-.01)

with open('../../data/test/sphere-cf2-theta016755-inside.csv', 'w') as w:
    w.write('X,Y,Z,Grain ID\n')
    for index in indices:
        x_, y_, z_ = kd_tree.data[index]
        grain_id = grain_ids[index]
        w.write('{},{},{},{}\n'.format(x_,y_,z_,grain_id%10000))
        
indices = kd_tree.query_ball_point((x,y,z),actual_distance+.01)

with open('../../data/test/sphere-cf2-theta016755-outside.csv', 'w') as w:
    w.write('X,Y,Z,Grain ID\n')
    for index in indices:
        x_, y_, z_ = kd_tree.data[index]
        grain_id = grain_ids[index]
        w.write('{},{},{},{}\n'.format(x_,y_,z_,grain_id%10000))